## LighFM

M. Pontier vous contact pour l'aider à construire un système de recommandation. Il dispose d'une base de données comportant des données concernant ses utilisateurs (anonymisé) contenant les artistes qu'ils écoutent sur sa plateforme ainsi que le nombre d'écoutes.***Monsieur pontier souhaite recommander à ses utilisateur des artistes qu'il n'ont pas encore écoutés, et cela en fonction de leurs préférences musicale.***

Monsieur pontier souhaite utiliser la librairie Lightfm, avec laquelle il a déjà un driver permettant de charger ses données qu'il vous fournit, un vrai bonus. Monsieur Pontier à pu voir que la documentation comporte plusieurs modèle, ***il souhaite évaluer les modèle sur une jeux de train/test et utiliser le meilleurs modéle.***

***Pour l'évaluation, il souhaite comparer la mesure AUC, la précision et le rappel (visiter la documentation de Lightfm), qui devront être présenté dans un tableau.***

Bonus 1
Comparer les résulats de l'AUC avec le meilleurs modéle de lightfm et une PCA (TruncatedSDV).

Bonus 2
L'apprentissage devant être le plus rapide possible tout en obtenant les meilleurs résultats, il vous est demandé de trouver le nombre d'itération permettant d'atteindre la convergence de 95% de la valeur maximal d'AUC sur le jeux de test.

Veille
Quelle système de recommandation allez vous mettre en place ?

Qu'est ce que Lightfm ?

Qu'est ce un système de recommandation dit à "implicit feedback" ? Et a "explicit feedback ?

Ressources:
LightFM: https://github.com/lyst/lightfm

Jeux de données Last.fm : https://grouplens.org/datasets/hetrec-2011/

The world is large and we know just a small part of it, dont forget the big picture: https://github.com/jihoo-kim/awesome-RecSys

https://towardsdatascience.com/solving-business-usecases-by-recommender-system-using-lightfm-4ba7b3ac8e62

In [4]:
import pandas as pd
import numpy as np
from lightfm import LightFM

data = pd.read_csv('doc.csv')

ratings_df = data.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings = ratings_df.fillna(0).values
ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00115397, ..., 0.        , 0.        ,
        0.        ]])

In [5]:
from scipy.sparse import csr_matrix

# Build a sparse matrix
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape # n_items est le nombre d'artistes et n_users = nbre de pers
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = data.sort_values("artistID")["name"].unique()


rating matrix shape (1892, 17632)


In [6]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
Xcoo = X.tocoo() # scipy = plusieurs form
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions)

In [7]:
# Train
model = LightFM(learning_rate=0.05, loss='warp',learning_schedule='adadelta')
model.fit(train, epochs=10, num_threads=2)

In [8]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))




Precision: train 0.36, test 0.13.
AUC: train 0.96, test 0.87.


In [9]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)


['The Killers' 'New Order' 'Radiohead' ... 'Motörhead' 'Destruction'
 'Testament']


In [12]:
def get_recommendations(userID):
    scores = model.predict(userID, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    return top_items

get_recommendations(1642)

array(['Ke$ha', 'David Guetta', 'Avril Lavigne', ...,
       'Александр Панайотов', 'Chris Dair', 'Asher Book'], dtype=object)

In [14]:
# Optimisation de paramètres avec GridSearch

from sklearn.model_selection import ParameterGrid# Create the parameter grid based on the results of random search 

param_grid = {
    'learning_rate': [0.05 , 0.08],
    'learning_schedule':['adagrad','adadelta'],
    'loss': ['warp','bpr','logistic','warp-kos']
    
}
# definition liste score auc
auc_score_values = []

for grid in ParameterGrid(param_grid):
    model = LightFM(**grid)
    pred = model.fit(train)
    auc_score_values.append(round(auc_score(model, test, train_interactions=train).mean(),3))
    
max_value = max(auc_score_values) 
max_index = np.argmax(auc_score_values)
ParameterGrid(param_grid)[max_index ].items()

dict_items([('loss', 'warp'), ('learning_schedule', 'adadelta'), ('learning_rate', 0.05)])